In [442]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [443]:
%store -r df_test_passenger_id_list 
df_test_passenger_id_list = df_test_passenger_id_list
df = pd.read_csv('../data/data_before_preprocessing.csv')
df.head()

,PassengerId,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,passenger_number_in_group,total_luxury_amenities,deck,num,side,first name,last name,group_size,HomePlanet,cabin_number_distribution_chunk
0,0001_01,False,TRAPPIST-1e,39,False,0.0,0.0,0.0,0.0,0.0,...,1,0.0,B,0,P,Maham,Ofracculy,1,Europa,chunk1
1,0002_01,False,TRAPPIST-1e,24,False,109.0,9.0,25.0,549.0,44.0,...,1,736.0,F,0,S,Juanna,Vines,1,Earth,chunk1
2,0003_01,False,TRAPPIST-1e,58,True,43.0,3576.0,0.0,6715.0,49.0,...,1,10383.0,A,0,S,Altark,Susent,2,Europa,chunk1
3,0003_02,False,TRAPPIST-1e,33,False,0.0,1283.0,371.0,3329.0,193.0,...,2,5176.0,A,0,S,Solam,Susent,2,Europa,chunk1
4,0004_01,False,TRAPPIST-1e,16,False,303.0,70.0,151.0,565.0,2.0,...,1,1091.0,F,1,S,Willy,Santantines,1,Earth,chunk1


In [444]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   PassengerId                      12970 non-null  object 
 1   CryoSleep                        12970 non-null  bool   
 2   Destination                      12970 non-null  object 
 3   Age                              12970 non-null  int64  
 4   VIP                              12970 non-null  bool   
 5   RoomService                      12970 non-null  float64
 6   FoodCourt                        12970 non-null  float64
 7   ShoppingMall                     12970 non-null  float64
 8   Spa                              12970 non-null  float64
 9   VRDeck                           12970 non-null  float64
 10  Name                             12676 non-null  object 
 11  Transported                      12970 non-null  bool   
 12  passenger_group   

In [445]:
df.drop(columns=['VIP', 'Name', 'last name', 'first name', 'num', 'passenger_group'], inplace=True)

In [446]:
df['CryoSleep'] = df['CryoSleep'].astype(int)


In [447]:
df.columns

Index(['PassengerId', 'CryoSleep', 'Destination', 'Age', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Transported',
       'passenger_number_in_group', 'total_luxury_amenities', 'deck', 'side',
       'group_size', 'HomePlanet', 'cabin_number_distribution_chunk'],
      dtype='object')

In [448]:
X_test = df[df['PassengerId'].isin(df_test_passenger_id_list)]
test_passenger_id = X_test['PassengerId']
test_passenger_id

8693     0013_01
8694     0018_01
8695     0019_01
8696     0021_01
8697     0023_01
          ...   
12965    9266_02
12966    9269_01
12967    9271_01
12968    9273_01
12969    9277_01
Name: PassengerId, Length: 4277, dtype: object

In [449]:
X_test = X_test.drop(columns = ['PassengerId'])

In [450]:
X_test.shape

(4277, 16)

In [451]:
df_train = df[~df['PassengerId'].isin(df_test_passenger_id_list)]

In [452]:
df_train

,PassengerId,CryoSleep,Destination,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,passenger_number_in_group,total_luxury_amenities,deck,side,group_size,HomePlanet,cabin_number_distribution_chunk
0,0001_01,0,TRAPPIST-1e,39,0.0,0.0,0.0,0.0,0.0,False,1,0.0,B,P,1,Europa,chunk1
1,0002_01,0,TRAPPIST-1e,24,109.0,9.0,25.0,549.0,44.0,True,1,736.0,F,S,1,Earth,chunk1
2,0003_01,0,TRAPPIST-1e,58,43.0,3576.0,0.0,6715.0,49.0,False,1,10383.0,A,S,2,Europa,chunk1
3,0003_02,0,TRAPPIST-1e,33,0.0,1283.0,371.0,3329.0,193.0,False,2,5176.0,A,S,2,Europa,chunk1
4,0004_01,0,TRAPPIST-1e,16,303.0,70.0,151.0,565.0,2.0,True,1,1091.0,F,S,1,Earth,chunk1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,55 Cancri e,41,0.0,6819.0,0.0,1643.0,74.0,False,1,8536.0,A,P,1,Europa,chunk1
8689,9278_01,1,PSO J318.5-22,18,0.0,0.0,0.0,0.0,0.0,False,1,0.0,G,S,1,Earth,chunk2
8690,9279_01,0,TRAPPIST-1e,26,0.0,0.0,1872.0,1.0,0.0,True,1,1873.0,G,S,1,Earth,chunk2
8691,9280_01,0,55 Cancri e,32,0.0,1049.0,0.0,353.0,3235.0,False,1,4637.0,E,S,2,Europa,chunk2


In [453]:
X = df_train.drop(columns=['Transported', 'PassengerId'])
y = df_train['Transported']

In [454]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [455]:
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

In [456]:
categ_cols = ['Destination', 'HomePlanet', 'deck', 'side', 'cabin_number_distribution_chunk']
categories_enc = encoder.fit_transform(X_train[categ_cols])
categories_enc_df = pd.DataFrame(categories_enc, columns = encoder.get_feature_names_out(categ_cols), index=X_train.index)

In [457]:
X_train.drop(columns=categ_cols, inplace=True)
X_train = pd.concat([X_train, categories_enc_df], axis = 1)


In [458]:
categories_enc_val = encoder.transform(X_val[categ_cols])

# Перетворюємо у DataFrame
categories_enc_val_df = pd.DataFrame(
    categories_enc_val,
    columns=encoder.get_feature_names_out(categ_cols),
    index=X_val.index
)

In [459]:
X_val.drop(columns=categ_cols, inplace=True)
X_val = pd.concat([X_val, categories_enc_val_df], axis = 1)

In [460]:
# Для тесту
categories_enc_test = encoder.transform(X_test[categ_cols])

# Перетворюємо у DataFrame
categories_enc_test_df = pd.DataFrame(
    categories_enc_test,
    columns=encoder.get_feature_names_out(categ_cols),
    index=X_test.index
)

In [461]:
X_test.drop(columns=categ_cols, inplace=True)
X_test = pd.concat([X_test, categories_enc_test_df], axis = 1)

In [462]:
expenses= ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck', 'total_luxury_amenities']
X_train[expenses] = np.log1p(X_train[expenses])
X_val[expenses] = np.log1p(X_val[expenses])
X_test[expenses] = np.log1p(X_test[expenses])

# Scaling

In [463]:
scaler = StandardScaler()

In [464]:
numerical_columns = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa',
       'VRDeck', 'passenger_number_in_group', 'total_luxury_amenities',
       'group_size'] 

In [465]:
X_train_scaled = scaler.fit_transform(X_train[numerical_columns])
X_val_scaled = scaler.transform(X_val[numerical_columns])
X_test_scaled = scaler.transform(X_test[numerical_columns])

In [466]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=numerical_columns, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=numerical_columns, index=X_val.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=numerical_columns, index=X_test.index)

In [467]:
X_train = X_train.drop(columns=numerical_columns).join(X_train_scaled)
X_val = X_val.drop(columns=numerical_columns).join(X_val_scaled)
X_test = X_test.drop(columns=numerical_columns).join(X_test_scaled)

In [468]:
X_test.info

<bound method DataFrame.info of        CryoSleep  Transported  Destination_PSO J318.5-22  \
8693           1         True                        0.0   
8694           0         True                        0.0   
8695           1         True                        0.0   
8696           0         True                        0.0   
8697           0         True                        0.0   
...          ...          ...                        ...   
12965          1         True                        0.0   
12966          0         True                        0.0   
12967          1         True                        0.0   
12968          0         True                        0.0   
12969          1         True                        1.0   

       Destination_TRAPPIST-1e  HomePlanet_Europa  HomePlanet_Mars  deck_B  \
8693                       1.0                0.0              0.0     0.0   
8694                       1.0                0.0              0.0     0.0   
8695         

In [469]:
%store X_train
%store X_val
%store X_test

%store y_train
%store y_val
%store test_passenger_id

Stored 'X_train' (DataFrame)
Stored 'X_val' (DataFrame)
Stored 'X_test' (DataFrame)
Stored 'y_train' (Series)
Stored 'y_val' (Series)
Stored 'test_passenger_id' (Series)
